# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

[**Introduction**](#introduction)

* Many People in their vacations choose famous gambling places like Los Vegas or Los-
  Angeles which hosts lots of Casino’s.The Purpose of this project is to provide a quick insite of
  Neighbourhood Casino's in these famous gambling destinations.

[**BussinessProblem**](#BussinessProblem)
* To Compare Neighbourhood’s of Casino’s for both Nevada and California using Four
  Square API and Google maps API.
*  To Provide a Quick Insight on the Famous Gambling destinations.
*  To Compare the Hotels in both destinations

[**Data**](#Data)
* There is primarily one source for the data containing list of casino’s, which is Wikipedia.
* For extracting data from Wikipedia for list of casino’s and their respective states a famous python
  library for WebScraping (BeautifulSoup) is used.
* For Extracting Casino's related information we are using first two links and Four Square API for displaying casino on map.
* Later, for extracting hotels in the two destinations Google API  is used to get the latitudes and
  Longitudes.
* **Source:**
1. https://en.wikipedia.org/wiki/List_of_casinos_in_Nevada
2. https://en.wikipedia.org/wiki/List_of_casinos_in_California
3. https://en.wikipedia.org/wiki/List_of_counties_in_Nevada
4. https://en.wikipedia.org/wiki/List_of_largest_California_cities_by_population

[**Target Audience**](#TargetAudience)

1. Gamblers
2. Tourists

In [1]:
from bs4 import BeautifulSoup
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from IPython.display import Image,display,Markdown 
from IPython.core.display import HTML
import re
display(Markdown(f'__Importing Modules for Map visualizations__'))

__Importing Modules for Map visualizations__

In [2]:
# !conda install -c conda-forge geopy --yes 
# libraries for displaying images
# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
display(Markdown(f'__A Method for extracting geo location data__'))

__A Method for extracting geo location data__

In [3]:
def geo_locator(address1):
    geolocator = Nominatim(user_agent="foursquare_agent",timeout=3)
    try:
        location = geolocator.geocode(address1)
        latitude = location.latitude
        longitude = location.longitude
    except AttributeError:
        print('{} has No Latitudes and Longitudes'.format(address1)) 
        latitude = 'NA'
        longitude='NA'
        pass
    return latitude, longitude
display(Markdown(f'__Providing Four square API credential details__'))

__Providing Four square API credential details__

In [4]:
CLIENT_ID = 'GB13T5YS5OMPQULTOQRKEUAO1QPGVW2FZU2RMENZUQQ4XIEH' # your Foursquare ID
# CLIENT_SECRET = 'Hidden' # your Foursquare Secret
# VERSION = '20180604'
VERSION = '20191225'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
display(Markdown(f'__Scrapng Wikipedia page for California casino data__'))
display(Markdown(f'__Filtering rows which are not relevent__'))

Your credentails:
CLIENT_ID: GB13T5YS5OMPQULTOQRKEUAO1QPGVW2FZU2RMENZUQQ4XIEH
CLIENT_SECRET:N0ASGQHHQDUMGRTDD0ZFDEBJUDLGN5DEU44VXESRIS1W1RZX


__Scrapng Wikipedia page for California casino data__

__Filtering rows which are not relevent__

**Analysis**
========

In [5]:
# Web Scraping casion's List
casino_angeles = requests.get('https://en.wikipedia.org/wiki/List_of_casinos_in_California')
angeles_soup = BeautifulSoup(casino_angeles.content,'html5lib')
table_angeles = angeles_soup.find("table",{"class":"toccolours sortable"})
for tag in table_angeles.find_all('a',title = re.compile("\(page does not exist\)")):
    tag.decompose()
display(Markdown(f"__Creating a pandas dataframe with scraped data__"))
display(Markdown(f'__Renaming columns and dropping rows with null  values__'))

__Creating a pandas dataframe with scraped data__

__Renaming columns and dropping rows with null  values__

In [6]:
angeles_df = pd.read_html(str(table_angeles))[0]
angeles_df = angeles_df.drop(columns=['District','Comments']).sort_values(['Casino']).reset_index(drop=True)
angeles_df.dropna(subset=['Casino','County'],inplace=True)
display(angeles_df.head())
display(Markdown(f'__Getting the highly populated cities in california__'))

,Casino,City,County,State,Type
0,Agua Caliente Casino,Rancho Mirage,Riverside,California,Native American
1,Artichoke Joe's Casino,San Bruno,San Mateo,California,Card room
3,Bay 101,San Jose,Santa Clara,California,Card room
4,Black Oak Casino,Tuolumne,Tuolumne,California,Native American
5,Cache Creek Casino Resort,Brooks,Yolo,California,Native American


__Getting the highly populated cities in california__

In [7]:
top_city_calf = pd.read_html('https://en.wikipedia.org/wiki/List_of_largest_California_cities_by_population')[0]['City'].values
angeles_df = angeles_df[angeles_df.City.isin(top_city_calf)].reset_index(drop=True)
display(Markdown(f'__Scaripng the nevada related casino from wikepidia__'))

__Scaripng the nevada related casino from wikepidia__

In [8]:
# Web Scraping casion's List
casino_vegas = requests.get('https://en.wikipedia.org/wiki/List_of_casinos_in_Nevada')
vegas_soup = BeautifulSoup(casino_vegas.content,'html5lib')
table_vegas = vegas_soup.find("table",{"class":"toccolours sortable"})
for tag in table_vegas.find_all('a',title = re.compile("\(page does not exist\)")):
    tag.decompose()
display(Markdown(f'__Filtering and cleaning the data of nevada casino to make a dataframe__'))

__Filtering and cleaning the data of nevada casino to make a dataframe__

In [9]:
vegas_df = pd.read_html(str(table_vegas))[0]
vegas_df = vegas_df.drop(columns=['District[1]','Comments','Type']).sort_values(['Casino']).reset_index(drop=True)
vegas_df.dropna(subset=['Casino'],inplace=True)
display(Markdown(f'__Extracting the top vegas cities__'))
# vegas_df = vegas_df.loc[:30]
# vegas_df

__Extracting the top vegas cities__

In [10]:
vegas_cities = requests.get('https://en.wikipedia.org/wiki/List_of_counties_in_Nevada')
cities_soup = BeautifulSoup(vegas_cities.content,'html5lib')
table_cities = cities_soup.find("table",{"class":"wikitable sortable"})
for tag in table_cities.find_all('a',title = re.compile("\(page does not exist\)")):
    tag.decompose()
top_city_nev = pd.read_html(str(table_cities))[0].sort_values(['Population[5]'],ascending=False)['County seat[3]'][:1].values
vegas_df = vegas_df[vegas_df.City.isin(top_city_nev)].reset_index(drop=True)
display(Markdown(f'__Extracting latitude and longitude from california casino data__'))

__Extracting latitude and longitude from california casino data__

In [11]:
for n, val in enumerate(angeles_df['Casino'].values):
    angeles_df.loc[n,'Latitude'],angeles_df.loc[n,'Longitude'] = geo_locator(val)
display(Markdown(f'__Extracting latitude and longitude from Vegas casino data__'))

__Extracting latitude and longitude from Vegas casino data__

In [12]:
for n, val in enumerate(vegas_df['Casino'].values):
    vegas_df.loc[n,'Latitude'],vegas_df.loc[n,'Longitude'] = geo_locator(val)
display(Markdown(f'__Dropping null values in latitudes and longitudes of cities__'))

Castaways Hotel and Casino has No Latitudes and Longitudes
Holy Cow Casino and Brewery has No Latitudes and Longitudes
Lucky Dragon Hotel and Casino has No Latitudes and Longitudes
Pioneer Club Las Vegas has No Latitudes and Longitudes


__Dropping null values in latitudes and longitudes of cities__

In [13]:
angeles_df = angeles_df.drop(index=angeles_df[(angeles_df.Latitude == 'NA') & (angeles_df.Longitude == 'NA')].index).reset_index(drop=True)

/home/p/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [14]:
vegas_df = vegas_df.drop(index=vegas_df[(vegas_df.Latitude == 'NA') & (vegas_df.Longitude == 'NA')].index).reset_index(drop=True)

In [15]:
# api_key = hidden
radius = 50000
def get_coordinates(api_key, lat,long,radius,search_key):
    try:  
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&key={}&radius={}&type={}'\
               .format(lat,long,api_key,radius,search_key)
        response = requests.get(url).json()
#         results = response['results']['geometry']['location']
#         geographical_data = results[0]['geometry']['location'] # get geographical coordinates
#         lat = geographical_data['lat']
#         lon = geographical_data['lng']
#         return [lat, lon]
        return response
    except:
        return [None, None]
    

In [16]:
california_lat,california_long = geo_locator('California')
nevada_lat,nevada_long = geo_locator('Las Vegas')

In [17]:
california_hotel_list = []
hotel_list = get_coordinates(api_key,california_lat,california_long,50000,'Hotels')['results']
for hotel in hotel_list:
    california_hotel_list.append((hotel['geometry']['location']['lat'],hotel['geometry']['location']['lng'],\
                                  hotel['name'][:10]))

In [18]:
california_park_list = []
parks_list = get_coordinates(api_key,california_lat,california_long,50000,'Parks')['results']
for park in parks_list:
    california_park_list.append((park['geometry']['location']['lat'],\
                                 park['geometry']['location']['lng'],park['name'][:10]))

In [19]:
nevada_hotel_list = []
hotel_list = get_coordinates(api_key,nevada_lat,nevada_long,50000,'Hotels')['results']
for hotel in hotel_list:
    nevada_hotel_list.append((hotel['geometry']['location']['lat'],hotel['geometry']['location']['lng'],\
                                  hotel['name'][:10]))

In [20]:
nevada_park_list = []
parks_list = get_coordinates(api_key,nevada_lat,nevada_long,50000,'Parks')['results']
for park in parks_list:
    nevada_park_list.append((park['geometry']['location']['lat'],park['geometry']['location']['lng'],\
                                  park['name'][:10]))

In [21]:
nevada_map = folium.Map(location=[nevada_lat, nevada_long], zoom_start=8)
california_map = folium.Map(location=[california_lat, california_long], zoom_start=8)
display(Markdown(f'__Displaying the Nevada map with casino if any using foursquareAPI__'))

__Displaying the Nevada map with casino if any using foursquareAPI__

**Results and Discussion**
====
1. **Nevada** 
    * **Orange** circles denote casions
    * **Green** circles denote hotels
    * **Blue** circles denote parks
2. **California**
    * **Red** Circles denote casinos
    * **Green** Circles denote hotels
    * **Pink** Circles denote parks

In [22]:
'''
We can observe most of the Casino are concentrated over Vegas
'''
display(Markdown(f'__Displaying the Nevada map with casino if any using foursquareAPI__'))
for lat,long in zip(vegas_df['Latitude'],vegas_df['Longitude']):
# add a red circle marker to represent the Conrad Hotel
    folium.features.CircleMarker(
        [float(lat), float(long)],
        radius=3,
        color='orange',
        popup='Casino',
        fill = True,
        fill_color = 'pink',
        fill_opacity = 0.6,
    ).add_to(nevada_map)
nevada_map

__Displaying the Nevada map with casino if any using foursquareAPI__

In [23]:
display(Markdown(f'__Displaying the California map with casino if any using foursquareAPI__'))
for lat,long in zip(angeles_df['Latitude'],angeles_df['Longitude']):
# add a red circle marker to represent the Conrad Hotel
    folium.features.CircleMarker(
        [float(lat), float(long)],
        radius=3,
        color='red',
        popup='casino',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6,
    ).add_to(california_map)
california_map

__Displaying the California map with casino if any using foursquareAPI__

In [24]:
display(Markdown(f'__Displaying the California map with hotels if any using Google API__'))
for hotel in california_hotel_list:
# add a red circle marker to represent the Conrad Hotel
    folium.features.CircleMarker(
        [float(hotel[0]), float(hotel[1])],
        radius=3,
        color='green',
        popup=str(hotel[2]),
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6,
    ).add_to(california_map)
california_map

__Displaying the California map with hotels if any using Google API__

In [25]:
display(Markdown(f'__Displaying the Nevada map with hotels if any using Google API__'))
for hotel in nevada_hotel_list:
# add a red circle marker to represent the Conrad Hotel
    folium.features.CircleMarker(
        [float(hotel[0]), float(hotel[1])],
        radius=3,
        color='green',
        popup=str(hotel[2]),
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6,
    ).add_to(nevada_map)
nevada_map

__Displaying the Nevada map with hotels if any using Google API__

In [26]:
display(Markdown(f'__Displaying the California map with parks if any using Google API__'))
for park in california_park_list:
# add a red circle marker to represent the Conrad Hotel
    folium.features.CircleMarker(
        [float(park[0]), float(park[1])],
        radius=3,
        color='blue',
        popup=str(park[2]),
        fill = True,
        fill_color = 'yellow',
        fill_opacity = 0.6,
    ).add_to(california_map)
california_map

__Displaying the California map with parks if any using Google API__

In [27]:
display(Markdown(f'__Displaying the Nevada map with many parks if any using Google API__'))
for park in nevada_park_list:
# add a red circle marker to represent the Conrad Hotel
    folium.features.CircleMarker(
        [float(park[0]), float(park[1])],
        radius=3,
        color='pink',
        popup=str(park[2]),
        fill = True,
        fill_color = 'grey',
        fill_opacity = 0.6,
    ).add_to(nevada_map)
nevada_map

__Displaying the Nevada map with many parks if any using Google API__

**Conclusion**
==========
* In Nevada Casino's are more concentrated at Las Vegas.
* There are many hotels in las vegas.
* In California casino's are spread at four locations(San Jose,Temecula,Los Angeles and Indio).
* In California Parks and hotels are concentrated away from highly populated areas of california like Los Angeles.

* **For person to whom gambling is a hobby  and doesn't care about other eminities vegas is a right fit.**
* **For a person to whom travelling is fun california would be a right pick.**
